In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.feature_selection import SelectKBest, f_classif

In [ ]:
import pandas as pd

# carregando o dataset
df = pd.read_csv('/content/creditcard.csv')

# informações básicas sobre o dataset
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1759 entries, 0 to 1758
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    1759 non-null   float64
 1   V1      1759 non-null   float64
 2   V2      1759 non-null   float64
 3   V3      1759 non-null   float64
 4   V4      1759 non-null   float64
 5   V5      1759 non-null   float64
 6   V6      1759 non-null   float64
 7   V7      1759 non-null   float64
 8   V8      1759 non-null   float64
 9   V9      1759 non-null   float64
 10  V10     1759 non-null   float64
 11  V11     1759 non-null   float64
 12  V12     1759 non-null   float64
 13  V13     1759 non-null   float64
 14  V14     1759 non-null   float64
 15  V15     1759 non-null   float64
 16  V16     1759 non-null   float64
 17  V17     1759 non-null   float64
 18  V18     1759 non-null   float64
 19  V19     1759 non-null   float64
 20  V20     1759 non-null   float64
 21  V21     1759 non-null   float64
 22  

In [ ]:
# definindo as variáveis independentes (X) e a variável dependente (y)
X = df.drop(columns='Class')  # supondo que 'Class' seja a coluna de transação fraudulenta (1 para fraude, 0 para normal)
y = df['Class']

# dividindo o dataset em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# treinando e avaliando o modelo com todos os atributos
clf = DecisionTreeClassifier(random_state=1)
clf.fit(X_train, y_train)
y_pred_all = clf.predict(X_test)

print("Desempenho com todos os atributos:")
print("Acurácia:", accuracy_score(y_test, y_pred_all))
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred_all))
print("F1 Score:", f1_score(y_test, y_pred_all))

Desempenho com todos os atributos:
Acurácia: 0.9113636363636364
Matriz de Confusão:
[[298  16]
 [ 23 103]]
F1 Score: 0.8408163265306122


In [ ]:
# seleção de atributos com SelectKBest
selector = SelectKBest(f_classif, k=10)  # selecionando os 10 atributos mais relevantes
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# treinando e avaliando o modelo com os atributos selecionados antes do tuning
clf = DecisionTreeClassifier(random_state=1)
clf.fit(X_train_selected, y_train)
y_pred_selected = clf.predict(X_test_selected)

print("\nDesempenho com atributos selecionados (antes do tuning):")
print("Acurácia:", accuracy_score(y_test, y_pred_selected))
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred_selected))
print("F1 Score:", f1_score(y_test, y_pred_selected))


Desempenho com atributos selecionados (antes do tuning):
Acurácia: 0.9318181818181818
Matriz de Confusão:
[[306   8]
 [ 22 104]]
F1 Score: 0.8739495798319328


In [ ]:
# tuning de hiperparâmetros com GridSearchCV
# definindo o espaço de hiperparâmetros para a árvore de decisão
param_grid = {
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 5, 10]
}

# usando GridSearchCV para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=1),
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1
)

# realizando a busca com os dados de treino (usando atributos selecionados)
grid_search.fit(X_train_selected, y_train)

# exibindo os melhores hiperparâmetros encontrados
print("\nMelhores hiperparâmetros:", grid_search.best_params_)

# treinando o modelo com os melhores hiperparâmetros
best_clf = grid_search.best_estimator_
best_clf.fit(X_train_selected, y_train)
y_pred_tuned = best_clf.predict(X_test_selected)

# avaliando o modelo ajustado
print("\nDesempenho com atributos selecionados e hiperparâmetros otimizados:")
print("Acurácia:", accuracy_score(y_test, y_pred_tuned))
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred_tuned))
print("F1 Score:", f1_score(y_test, y_pred_selected))


Melhores hiperparâmetros: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10}

Desempenho com atributos selecionados e hiperparâmetros otimizados:
Acurácia: 0.9386363636363636
Matriz de Confusão:
[[308   6]
 [ 21 105]]
F1 Score: 0.8739495798319328
